In [ ]:
!pip install wikipedia
!pip install transformers
!pip install keybert

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/datasets/AIIJC/2-NLP/train_data.csv")[:50000]
test = pd.read_csv("/content/drive/MyDrive/datasets/AIIJC/2-NLP/test.csv")
sample = pd.read_csv("/content/drive/MyDrive/datasets/AIIJC/2-NLP/sample_submission.csv")

In [4]:
from transformers import pipeline
from keybert import KeyBERT
import wikipedia as wiki
from tqdm.notebook import tqdm
import warnings
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer


class QuestionAnswering():
    def __init__(self, train_data, transformer_name='bert-large-uncased-whole-word-masking-finetuned-squad', reserve_transformer='deepset/bert-large-uncased-whole-word-masking-squad2', min_kw_len=3, lang='en', ignore_warnings=True):
        if ignore_warnings: warnings.simplefilter("ignore")
        self.transformer = pipeline(task='question-answering', model=transformer_name, tokenizer=transformer_name)
        self.reserve_transformer = pipeline(task='question-answering', model=reserve_transformer, tokenizer=transformer_name)
        self.kw_model = KeyBERT()
        self.train = train_data
        self.min_kw_len = min_kw_len
        self.vectorizer = TfidfVectorizer()
        self.vectorizer.fit(self.train['question'].tolist())
        self.corpus_v = self.vectorizer.transform(self.train['question'].tolist()).toarray()
        wiki.set_lang(lang)

    
    def __repr__(self):
        return "Question Answering class"


    def find_key_word(self, sentence, ngrams=(1,3)):
        return self.kw_model.extract_keywords(sentence, keyphrase_ngram_range=ngrams, stop_words=None)[0][0]


    def search_on_wiki(self, topic):
        page = wiki.page(''.join(topic.split()))
        return page.content

    
    def cosine_answer(self, question):
        text_v = self.vectorizer.transform([question]).toarray()
        max = {'value' : 0, 'id': 0}
        for i in range(len(self.corpus_v)):
            cos = 1 - cosine(text_v[0], self.corpus_v[i])
            if cos > max['value']:
                max['value'] = cos
                max['id'] = i

        return self.train['answer'][max['id']]


    def find_wiki_data(self, question):
        all_data = []
        keyw = []
        for i in range(self.min_kw_len, len(question.split())):
            try:
                keywords = self.find_key_word(question, (len(question.split()) - i-1, len(question.split()) - i))
                keyw.append(keywords)
                data = wiki.summary(keywords)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue

        for keywords in keyw:
            try:
                kw = wiki.search(keywords, results=1)
                data = wiki.summary(kw)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue

        for keywords in keyw:
            try:
                data = self.search_on_wiki(keywords)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue

        if len(all_data) > 0:
            return ' '.join(all_data)

        for keywords in keyw:
            try:
                kw = wiki.search(keywords, results=1)
                data = self.search_on_wiki(kw)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue

        keyw = []
        for i in list(range(0, self.min_kw_len))[::-1]:
            try:
                keywords = self.find_key_word(question, (len(question.split()) - i-1, len(question.split()) - i))
                keyw.append(keywords)
                data = wiki.summary(keywords)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue

        for keywords in keyw:
            try:
                data = self.search_on_wiki(keywords)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue

        for keywords in keyw:
            try:
                kw = wiki.search(keywords, results=1)
                data = wiki.summary(kw)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue

        for keywords in keyw:
            try:
                kw = wiki.search(keywords, results=1)
                data = self.search_on_wiki(kw)
                if len(data) > 0:
                    return data
                else:
                    continue
            except:
                continue
        
        for keywords in keyw:
            try:
                kws = wiki.search(keywords, results=20)
                for kw in kws:
                    data = self.search_on_wiki(kw)
                    if len(data) > 0:
                        return data
                    else:
                        continue
            except:
                continue

        return 0


    def give_answer(self, question):
        context = self.find_wiki_data(question)
        if context == 0: return self.cosine_answer(question)
        answer = self.transformer({
            'context' : context,
            'question' : question})['answer']
        if answer != '.': return answer
        else:
            answer = self.reserve_transformer({
                'context' : context,
                'question' : question})['answer']
            if answer != '.': return answer
            else:
                return self.cosine_answer(question)


    def predict(self, corpus):
        answers = []
        for question in tqdm(corpus):
            ans = self.give_answer(question)
            answers.append(ans)
        return answers

In [5]:
corpus = test['question'].tolist()

QA = QuestionAnswering(train)
# answers = QA.predict(corpus)

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import requests

requests.get("https://api.telegram.org/bot2057911206:AAGQfUTD03Vd0N6dhhJcgec1F-gMPmX0wqo/sendMessage?chat_id=799213094&text=Prediction is ready!")

<Response [200]>

In [ ]:
submission = sample.copy()
submission['answer'] = answers
submission.to_csv("submission_13.csv")

In [ ]:
submission = sample.copy()
submission['answer'] = answers
submission.to_csv("/content/drive/MyDrive/datasets/AIIJC/submission_13.csv")

In [6]:
!pip install speechrecognition 
!pip install pydub

     |████████████████████████████████| 32.8 MB 57 kB/s 


In [10]:
!pip install pytelegrambotapi

     |████████████████████████████████| 102 kB 5.8 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.1.1-py3-none-any.whl size=80779 sha256=3da5b107b729da56f8734837aa63dc7f1b7396ada71e6e2dc764d16c494829cc
  Stored in directory: /root/.cache/pip/wheels/00/9d/f5/f589ebef11a6541a4e5a1793a380db577bab5583eac6b45514
Successfully built pytelegrambotapi


In [ ]:
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
import telebot
from time import time
import os

%cd content
%rmdir voice_msgs
%mkdir voice_msgs


def get_large_audio_transcription(path):
    r = sr.Recognizer()
    sound = AudioSegment.from_ogg(path)  
    chunks = split_on_silence(sound,
        min_silence_len = 500,
        silence_thresh = sound.dBFS-14,
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                return 0
            else:
                text = f"{text.capitalize()}. "
                whole_text += text
    return whole_text

bot = telebot.TeleBot("2048868243:AAFnM9HA_kRW8_NYMjelhKbkQECsxP9Drpk")

@bot.message_handler(commands=['start'])
def start_handler(message):
    bot.send_message(message.chat.id, "Hi, it's QA bot. You can ask your questions :)")

@bot.message_handler(content_types=['text'])
def text_handler(message):
    answer = QA.give_answer(message.text)
    bot.send_message(message.chat.id, answer)

@bot.message_handler(content_types=['voice'])
def voice_processing(message):
    file_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open(f'voice_msgs/{message.chat.id}_{int(time())}.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
    name = f'voice_msgs/{message.chat.id}_{int(time())}.ogg'
    transcription = get_large_audio_transcription(name)
    if transcription == 0:
        bot.send_message(message.chat.id, 'Could\'t recognize your voice')
    else:
        transcription = transcription[:-2]+'?'
        bot.send_message(message.chat.id, f'Your voice was recognized as: {transcription}')
        answer = QA.give_answer(transcription)
        bot.send_message(message.chat.id, answer)

print('Bot started!')
bot.polling()

[Errno 2] No such file or directory: 'content'
/content
rmdir: failed to remove 'voice_msgs': Directory not empty
mkdir: cannot create directory ‘voice_msgs’: File exists
Bot started!
